In [1]:
# !pip install pytorch_lightning tensorboard

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import DataLoader
from emnist_prediction.datasets import HandwritingsDataset, HandwritingsBalancedDataset
import torch.nn.functional as F
import pytorch_lightning as pl
import os
from emnist_prediction.constants import CLASS_LABELS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from emnist_prediction.metrics import get_classification_report
from emnist_prediction.utils import get_classes_count

import shutil
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

DATA_DIR = Path('../data/input_data')
WRITING_DIR = Path('../data/fnn_models')
SUBDATA_DIR = Path("../data/input_data/subdata")

os.makedirs(SUBDATA_DIR, exist_ok=True)

for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


..\data\input_data\sample_submission.csv
..\data\input_data\X_test.npy
..\data\input_data\X_train.npy
..\data\input_data\X_val.npy
..\data\input_data\y_train.npy
..\data\input_data\y_val.npy
..\data\input_data\subdata\X_subtest.npy
..\data\input_data\subdata\X_subval.npy
..\data\input_data\subdata\y_subtest.npy
..\data\input_data\subdata\y_subval.npy


In [3]:
from emnist_prediction.utils import split_save

X_val = np.load(DATA_DIR / 'X_val.npy')
y_val = np.load(DATA_DIR / 'y_val.npy')
split_save(X_val, y_val, stratify=y_val, writing_dir=SUBDATA_DIR)

y_train = np.load(DATA_DIR / 'y_train.npy')

y_true = np.load(SUBDATA_DIR / 'y_subtest.npy')
y_true_labels = y_true.argmax(axis=-1)

In [4]:
batch_size = 32
max_epochs = 35
input_size = 784
n_classes=len(CLASS_LABELS)

In [7]:
from emnist_prediction.fnn import FNN
from emnist_prediction.utils import id_2_label, label_2_id, oneh_2_label, label_2_1hot, get_classes_count
from emnist_prediction.constants import IMG_SIZE, CLASS_LABELS
from emnist_prediction.transforms import image_transform, ImgTransform, Reshape
from torch.utils.data import DataLoader
from emnist_prediction.metrics import min_f1_score, get_classification_report
from sklearn.metrics import classification_report

### FNN

In [8]:
subtest_labels = np.load(SUBDATA_DIR / 'y_subtest.npy').argmax(axis=-1)

#### Approach 1: Train baseline model 



In [74]:
batch_size = 32
input_size = 784
n_classes=len(CLASS_LABELS)
hidden_sizes = [512, 256, 256, 128]

X_train = np.load(DATA_DIR / 'X_train.npy')

X_mean = X_train.mean()
X_std = X_train.std()

def normalize_with_train_stats(X):
    return (X - X_mean) / X_std

train_dataset = HandwritingsDataset(X_file_path=DATA_DIR / 'X_train.npy', y_file_path=DATA_DIR / 'y_train.npy',
                                             data_transform=normalize_with_train_stats)

train_loader = DataLoader(train_dataset, num_workers=4,
                         batch_size=batch_size, shuffle=True)


val_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subval.npy', SUBDATA_DIR / 'y_subval.npy',
                                             data_transform=normalize_with_train_stats)

val_loader = DataLoader(val_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

test_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subtest.npy', SUBDATA_DIR / 'y_subtest.npy',
                                             data_transform=normalize_with_train_stats)

test_loader = DataLoader(test_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

fnn_checkpoint_dir = WRITING_DIR / 'v1_fnn_4_layers_train_norm'

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=fnn_checkpoint_dir / 'best_results', filename='{epoch}-{val_loss:.2f}-{train_loss:.2f}', 
                                      monitor='val_loss', save_last=True) 
early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', verbose=True, patience=4)

fnn = FNN(input_size=input_size, n_classes=n_classes, hidden_sizes=hidden_sizes)

trainer = pl.Trainer(max_epochs=max_epochs, default_root_dir=fnn_checkpoint_dir, fast_dev_run=True,
                     callbacks=[early_stopping, checkpoint_callback])

trainer.fit(fnn, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory C:\Users\julia\PycharmProjects\EMNIST Prediction\data\fnn_models\v1_fnn_4_layers_train_norm\best_results exists and is not empty.

  | Name           | Type              | Params
-----------------------------------------------------
0 | train_f1_score | MulticlassF1Score | 0     
1 | val_f1_score   | MulticlassF1Score | 0     
2 | linears        | ModuleList        | 635 K 
-----------------------------------------------------
635 K     Trainable params
0         Non-trainable params
635 K     Total params
2.541     Total estimated model params size (MB)
c:\Users\julia\anaconda3\envs\Da

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
`Trainer.fit` stopped: `max_steps=1` reached.


In [75]:
fnn = FNN.load_from_checkpoint(fnn_checkpoint_dir / 'best_results/epoch=20-val_loss=0.40-train_loss=0.27.ckpt', input_size=input_size,
                               n_classes=n_classes, hidden_sizes=hidden_sizes)

trainer = pl.Trainer()
predictions = trainer.predict(fnn, test_loader)

print(classification_report(subtest_labels, torch.cat(predictions).numpy()))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.36      0.94      0.52       320
           1       0.00      0.00      0.00       194
           2       0.98      0.97      0.98       505
           3       0.89      0.86      0.88       228
           4       0.96      0.95      0.95       247
           5       0.92      0.98      0.95       459
           6       0.00      0.00      0.00       126
           7       0.90      0.96      0.93       158
           8       0.96      0.98      0.97       597
           9       0.86      0.93      0.89       188
          10       0.00      0.00      0.00       124
          11       0.94      0.98      0.96       254
          12       0.98      0.99      0.98       450
          13       0.95      0.96      0.95       412
          14       0.97      0.98      0.97      1249
          15       0.97      0.95      0.96       417
          16       0.96      0.92      0.94       130
          17       0.95    

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [79]:
clf_report = get_classification_report(subtest_labels, torch.cat(predictions).numpy())
clf_report = clf_report.rename(columns={'f1_score': 'f1_score_baseline'})
clf_report.sort_values(by=['f1_score_baseline'])

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1_score_baseline,support
B,0.000000,0.000000,0.000000,194
X,0.000000,0.000000,0.000000,138
G,0.000000,0.000000,0.000000,126
K,0.000000,0.000000,0.000000,124
A,0.355713,0.943750,0.516681,320
D,0.894977,0.859649,0.876957,228
J,0.861386,0.925532,0.892308,188
V,0.911894,0.892241,0.901961,232
H,0.904192,0.955696,0.929231,158
Y,0.935897,0.924051,0.929936,237


In [80]:
clf_report[clf_report['f1_score_baseline'] == 0.0].index

Index(['B', 'G', 'K', 'X'], dtype='object')

### This will be removed?
#### Approach 2: Try with normalizing accross features 

This would make each of 28x28 features normalized with mean=0 and std=1

In [42]:
batch_size = 32
input_size = 784
n_classes=len(CLASS_LABELS)
hidden_sizes = [512, 256, 256, 128]

X_train = np.load(DATA_DIR / 'X_train.npy')

X_mean = X_train.transpose([1, 2, 0]).mean(axis=-1)
X_std = X_train.transpose([1, 2, 0]).std(axis=-1)

assert X_mean.shape == (28, 28)

if type(X_std) == np.ndarray:
    X_std[X_std == 0] = 1

def normalize_with_train_stats(X):
    return (X - X_mean) / X_std

train_dataset = HandwritingsDataset(X_file_path=DATA_DIR / 'X_train.npy', y_file_path=DATA_DIR / 'y_train.npy',
                                             data_transform=normalize_with_train_stats)

train_loader = DataLoader(train_dataset, num_workers=4,
                         batch_size=batch_size, shuffle=True)


val_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subval.npy', SUBDATA_DIR / 'y_subval.npy',
                                             data_transform=normalize_with_train_stats)

val_loader = DataLoader(val_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

test_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subtest.npy', SUBDATA_DIR / 'y_subtest.npy',
                                             data_transform=normalize_with_train_stats)

test_loader = DataLoader(test_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

fnn_checkpoint_dir = WRITING_DIR / 'v1_fnn_4_layers_train_norm_features'

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=fnn_checkpoint_dir / 'best_results', filename='{epoch}-{val_loss:.2f}-{train_loss:.2f}', 
                                      monitor='val_loss', save_last=True) 

early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', verbose=True, patience=4)

fnn = FNN(input_size=input_size, n_classes=n_classes, hidden_sizes=hidden_sizes)
trainer = pl.Trainer(max_epochs=max_epochs, default_root_dir=fnn_checkpoint_dir, fast_dev_run=True,
                     callbacks=[early_stopping, checkpoint_callback])

trainer.fit(fnn, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory C:\Users\julia\PycharmProjects\EMNIST Prediction\data\fnn_models\v1_fnn_4_layers_train_norm_features\best_results exists and is not empty.

  | Name           | Type              | Params
-----------------------------------------------------
0 | train_f1_score | MulticlassF1Score | 0     
1 | val_f1_score   | MulticlassF1Score | 0     
2 | linears        | ModuleList        | 635 K 
-----------------------------------------------------
635 K     Trainable params
0         Non-trainable params
635 K     Total params
2.541     Total estimated model params size (MB)
c:\Users\julia\anaconda

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
`Trainer.fit` stopped: `max_steps=1` reached.


In [81]:
fnn = FNN.load_from_checkpoint(fnn_checkpoint_dir / 'best_results/epoch=22-val_loss=0.73-train_loss=0.61.ckpt', input_size=input_size,
                               n_classes=n_classes, hidden_sizes=hidden_sizes)
trainer = pl.Trainer()
predictions = trainer.predict(fnn, test_loader)

print(classification_report(subtest_labels, torch.cat(predictions).numpy()))

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/julia/PycharmProjects/EMNIST Prediction/data/fnn_models/v1_fnn_4_layers_train_norm/best_results/epoch=22-val_loss=0.73-train_loss=0.61.ckpt'

In [25]:
clf_report_norm_feat = get_classification_report(subtest_labels, torch.cat(predictions).numpy()).sort_values(by=['f1_score'])

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
clf_report_norm_feat

,precision,recall,f1_score,support
M,0.000000,0.000000,0.000000,450
B,0.000000,0.000000,0.000000,194
X,0.000000,0.000000,0.000000,138
W,0.000000,0.000000,0.000000,235
H,0.000000,0.000000,0.000000,158
A,0.023047,0.056250,0.032698,320
F,0.386736,0.978214,0.554321,459
G,0.942529,0.650794,0.769953,126
Q,0.855072,0.907692,0.880597,130
D,0.909091,0.877193,0.892857,228


In [27]:
clf_report_norm_feat[clf_report_norm_feat['f1_score'] == 0.0].index

Index(['M', 'B', 'X', 'W', 'H'], dtype='object')

In [32]:
clf_report_norm_feat['f1_score_global'] = clf_report_4l_norm['f1_score']
clf_report_norm_feat

,precision,recall,f1_score,support,f1_score_global
M,0.000000,0.000000,0.000000,450,0.984513
B,0.000000,0.000000,0.000000,194,0.000000
X,0.000000,0.000000,0.000000,138,0.000000
W,0.000000,0.000000,0.000000,235,0.941176
H,0.000000,0.000000,0.000000,158,0.929231
A,0.023047,0.056250,0.032698,320,0.516681
F,0.386736,0.978214,0.554321,459,0.948148
G,0.942529,0.650794,0.769953,126,0.000000
Q,0.855072,0.907692,0.880597,130,0.941176
D,0.909091,0.877193,0.892857,228,0.876957


So we'll go for global normalization

In [82]:
X_train = np.load(DATA_DIR / 'X_train.npy')

X_mean = X_train.mean()
X_std = X_train.std()

def normalize(X):
    return (X - X_mean) / X_std

In [83]:
batch_size = 32
input_size = 784
n_classes=len(CLASS_LABELS)
hidden_sizes = [512, 256, 256, 128]

train_dataset = HandwritingsDataset(X_file_path=DATA_DIR / 'X_train.npy', y_file_path=DATA_DIR / 'y_train.npy',
                                             data_transform=normalize)

train_loader = DataLoader(train_dataset, num_workers=4,
                         batch_size=batch_size, shuffle=True)


val_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subval.npy', SUBDATA_DIR / 'y_subval.npy',
                                             data_transform=normalize)

val_loader = DataLoader(val_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

test_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subtest.npy', SUBDATA_DIR / 'y_subtest.npy',
                                             data_transform=normalize)

test_loader = DataLoader(test_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

#### Approach 2 : Try with assigning class weights

Weights will be inversely proportional to the class count

In [84]:
y_train = np.load(DATA_DIR / 'y_train.npy')
classes_count = get_classes_count(y_train)
class_share = classes_count / sum(classes_count)
class_weights = 1 / (class_share * 100)
class_weights = class_weights.sort_index()
class_weights = torch.tensor(class_weights).to(device).type(torch.float32)

In [62]:
fnn_checkpoint_dir = WRITING_DIR / 'weighted_fnn_4_layers'

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=fnn_checkpoint_dir / 'best_results', filename='{epoch}-{val_loss:.2f}-{train_loss:.2f}', 
                                      monitor='val_loss', save_last=True) 
early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', verbose=True, patience=4)

fnn = FNN(input_size=input_size, n_classes=n_classes, hidden_sizes=hidden_sizes, class_weights=class_weights)
trainer = pl.Trainer(max_epochs=max_epochs, default_root_dir=fnn_checkpoint_dir, fast_dev_run=True, 
         callbacks=[early_stopping, checkpoint_callback])
# trainer.fit(fnn, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [87]:
fnn = FNN.load_from_checkpoint(fnn_checkpoint_dir / 'best_results/epoch=18-val_loss=0.35-train_loss=0.40.ckpt', input_size=input_size,
                               n_classes=n_classes, hidden_sizes=hidden_sizes, class_weights=class_weights)
trainer = pl.Trainer()
predictions = trainer.predict(fnn, test_loader)

print(classification_report(subtest_labels, torch.cat(predictions).numpy()))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.49      0.97      0.65       320
           1       0.93      0.87      0.90       194
           2       0.98      0.97      0.97       505
           3       0.00      0.00      0.00       228
           4       0.95      0.98      0.96       247
           5       0.96      0.95      0.96       459
           6       0.96      0.92      0.94       126
           7       0.00      0.00      0.00       158
           8       0.98      0.97      0.98       597
           9       0.84      0.97      0.90       188
          10       0.91      0.94      0.93       124
          11       0.92      0.98      0.95       254
          12       0.98      0.98      0.98       450
          13       0.93      0.95      0.94       412
          14       0.96      0.99      0.97      1249
          15       0.96      0.97      0.96       417
          16       0.90      0.95      0.92       130
          17       0.89    

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [88]:
clf_report_cls_weight = get_classification_report(subtest_labels, torch.cat(predictions).numpy()).sort_values(by=['f1_score'])

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
clf_report['class_weights'] = pd.Series(class_weights.numpy(), index=CLASS_LABELS)
clf_report['f1_score_class_weights'] = clf_report_cls_weight['f1_score']

Observations: assigning weights improves f1 score for the 2 minor classes, but there are still some 0's (for letters D and H). Smaller weights didn't affect much majority classes

In [90]:
clf_report[['f1_score_baseline', 'f1_score_class_weights', 'class_weights', 'support']].sort_values(by=['class_weights'])

,f1_score_baseline,f1_score_class_weights,class_weights,support
O,0.974217,0.973123,0.075636,1249
S,0.980049,0.982507,0.091004,1038
U,0.958991,0.956175,0.149951,630
I,0.970149,0.977330,0.158173,597
C,0.975025,0.974155,0.187202,505
T,0.973737,0.978328,0.192421,491
F,0.948148,0.956044,0.205808,459
M,0.984513,0.981132,0.209924,450
P,0.961353,0.964371,0.226364,417
N,0.954217,0.939904,0.229387,412


In [131]:
sum(clf_report['f1_score_class_weights'] > clf_report['f1_score_baseline'])

15

In [91]:
clf_report[clf_report['f1_score_class_weights'] == 0.0].index

Index(['D', 'H'], dtype='object')

#### Approach 3: Random oversampling, no image manipulation

Apply random oversampling method to make all classes counts equal

In [92]:
batch_size = 32
input_size = 784
n_classes=len(CLASS_LABELS)
hidden_sizes = [512, 256, 256, 128]

fnn_checkpoint_dir = WRITING_DIR / 'fnn_4_layers_rand_over_sample'

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=fnn_checkpoint_dir / 'best_results', filename='{epoch}-{val_loss:.2f}-{train_loss:.2f}', 
                                      monitor='val_loss') 

early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', verbose=True, patience=4)

fnn = FNN(input_size=input_size, n_classes=n_classes, hidden_sizes=hidden_sizes)
trainer = pl.Trainer(max_epochs=max_epochs, default_root_dir=fnn_checkpoint_dir,
                     callbacks=[early_stopping, checkpoint_callback], fast_dev_run=True)
trainer.fit(fnn, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory C:\Users\julia\PycharmProjects\EMNIST Prediction\data\fnn_models\fnn_4_layers_rand_over_sample\best_results exists and is not empty.

  | Name           | Type              | Params
-----------------------------------------------------
0 | train_f1_score | MulticlassF1Score | 0     
1 | val_f1_score   | MulticlassF1Score | 0     
2 | linears        | ModuleList        | 635 K 
-----------------------------------------------------
635 K     Trainable params
0         Non-trainable params
635 K     Total params
2.541     Total estimated model params size (MB)
c:\Users\julia\anaconda3\envs

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
`Trainer.fit` stopped: `max_steps=1` reached.


In [93]:
os.listdir(fnn_checkpoint_dir / 'best_results')

['epoch=4-val_loss=0.46-train_loss=0.33.ckpt']

In [94]:
fnn = FNN.load_from_checkpoint(fnn_checkpoint_dir / 'best_results/epoch=4-val_loss=0.46-train_loss=0.33.ckpt', input_size=input_size,
                               n_classes=n_classes, hidden_sizes=hidden_sizes)
trainer = pl.Trainer()
predictions = trainer.predict(fnn, test_loader)

print(classification_report(subtest_labels, torch.cat(predictions).numpy()))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.31      0.96      0.47       320
           1       0.97      0.92      0.94       194
           2       0.98      0.95      0.97       505
           3       0.85      0.88      0.86       228
           4       0.92      0.97      0.94       247
           5       0.95      0.95      0.95       459
           6       0.00      0.00      0.00       126
           7       0.92      0.91      0.92       158
           8       0.97      0.96      0.97       597
           9       0.83      0.96      0.89       188
          10       0.94      0.94      0.94       124
          11       0.96      0.95      0.96       254
          12       0.95      0.99      0.97       450
          13       0.92      0.97      0.95       412
          14       0.98      0.96      0.97      1249
          15       0.94      0.98      0.96       417
          16       0.85      0.98      0.91       130
          17       0.96    

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [95]:
clf_report_rand_ovsmpl = get_classification_report(subtest_labels, torch.cat(predictions).numpy()).sort_values(by=['f1_score'])

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [100]:
clf_report['f1_score_rand_balanced'] = clf_report_rand_ovsmpl['f1_score']

In [101]:
clf_report.sort_values(by=['support'])

,precision,recall,f1_score_baseline,support,class_weights,f1_score_class_weights,f1_score_rand_balanced
K,0.000000,0.000000,0.000000,124,0.765785,0.928571,0.943548
G,0.000000,0.000000,0.000000,126,0.750576,0.939271,0.000000
Q,0.960000,0.923077,0.941176,130,0.725365,0.921348,0.911032
Z,0.935252,0.962963,0.948905,135,0.699519,0.966790,0.945848
X,0.000000,0.000000,0.000000,138,0.681849,0.953405,0.948905
H,0.904192,0.955696,0.929231,158,0.599389,0.000000,0.917197
J,0.861386,0.925532,0.892308,188,0.502213,0.898765,0.886700
B,0.000000,0.000000,0.000000,194,0.487318,0.898396,0.941799
D,0.894977,0.859649,0.876957,228,0.414153,0.000000,0.863931
V,0.911894,0.892241,0.901961,232,0.407456,0.927350,0.844106


In [111]:
is_improved = (clf_report['f1_score_rand_balanced'] > clf_report['f1_score_baseline'])
is_improved.name = 'is improved'

In [117]:
clf_report[['support', 'f1_score_baseline', 'f1_score_rand_balanced']].join(is_improved).sort_values(by=['support'])

,support,f1_score_baseline,f1_score_rand_balanced,is improved
K,124,0.000000,0.943548,True
G,126,0.000000,0.000000,False
Q,130,0.941176,0.911032,False
Z,135,0.948905,0.945848,False
X,138,0.000000,0.948905,True
H,158,0.929231,0.917197,False
J,188,0.892308,0.886700,False
B,194,0.000000,0.941799,True
D,228,0.876957,0.863931,False
V,232,0.901961,0.844106,False


#### Approach 4: Increase network size 

In [118]:
hidden_sizes = [512, 512, 256, 256, 128]

fnn_checkpoint_dir = WRITING_DIR / 'fnn_5_layers_features_norm'

early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', verbose=True)

fnn = FNN(input_size=input_size, n_classes=n_classes, hidden_sizes=hidden_sizes)
trainer = pl.Trainer(max_epochs=max_epochs, default_root_dir=fnn_checkpoint_dir,
                     callbacks=[early_stopping], fast_dev_run=True)
trainer.fit(fnn, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.

  | Name           | Type              | Params
-----------------------------------------------------
0 | train_f1_score | MulticlassF1Score | 0     
1 | val_f1_score   | MulticlassF1Score | 0     
2 | linears        | ModuleList        | 897 K 
-----------------------------------------------------
897 K     Trainable params
0         Non-trainable params
897 K     Total params
3.592     Total estimated model params size (MB)
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-pack

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
`Trainer.fit` stopped: `max_steps=1` reached.


In [122]:
os.listdir(fnn_checkpoint_dir / 'lightning_logs/version_1/checkpoints')

['epoch=14-step=70860.ckpt']

In [123]:
fnn = FNN.load_from_checkpoint(fnn_checkpoint_dir / 'lightning_logs/version_1/checkpoints/epoch=14-step=70860.ckpt', input_size=input_size,
                               n_classes=n_classes, hidden_sizes=hidden_sizes)
trainer = pl.Trainer()
predictions = trainer.predict(fnn, test_loader)

print(classification_report(subtest_labels, torch.cat(predictions).numpy()))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.19      0.78      0.31       320
           1       0.59      0.74      0.66       194
           2       0.89      0.90      0.90       505
           3       0.93      0.58      0.72       228
           4       0.00      0.00      0.00       247
           5       0.73      0.73      0.73       459
           6       0.85      0.56      0.67       126
           7       0.00      0.00      0.00       158
           8       0.63      0.58      0.61       597
           9       0.74      0.53      0.62       188
          10       0.54      0.85      0.66       124
          11       0.88      0.73      0.80       254
          12       0.98      0.70      0.82       450
          13       0.80      0.81      0.80       412
          14       0.93      0.92      0.92      1249
          15       0.93      0.21      0.34       417
          16       0.91      0.62      0.74       130
          17       0.36    

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [124]:
clf_report_5_layers = get_classification_report(subtest_labels, torch.cat(predictions).numpy())

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [125]:
clf_report['f1_score_increased_size'] = clf_report_5_layers['f1_score']

In [129]:
clf_report.sort_values(by=['support'])

,precision,recall,f1_score_baseline,support,class_weights,f1_score_class_weights,f1_score_rand_balanced,f1_score_increased_size
K,0.000000,0.000000,0.000000,124,0.765785,0.928571,0.943548,0.658385
G,0.000000,0.000000,0.000000,126,0.750576,0.939271,0.000000,0.673077
Q,0.960000,0.923077,0.941176,130,0.725365,0.921348,0.911032,0.739726
Z,0.935252,0.962963,0.948905,135,0.699519,0.966790,0.945848,0.617211
X,0.000000,0.000000,0.000000,138,0.681849,0.953405,0.948905,0.563953
H,0.904192,0.955696,0.929231,158,0.599389,0.000000,0.917197,0.000000
J,0.861386,0.925532,0.892308,188,0.502213,0.898765,0.886700,0.619195
B,0.000000,0.000000,0.000000,194,0.487318,0.898396,0.941799,0.655963
D,0.894977,0.859649,0.876957,228,0.414153,0.000000,0.863931,0.716981
V,0.911894,0.892241,0.901961,232,0.407456,0.927350,0.844106,0.633053
